# <div style="text-align: center; background-color: #595964; font-family:Times New Roman; color: white; padding: 14px; line-height: 1;border-radius:20px">📊EDA | on Global University Rankings</div>

<h3 style="text-align: left;background-color: #00BFFF; font-family:Times New Roman; color: white; padding: 14px; line-height: 1; border-radius:10px"> About Dataset📁</h3>

<h4>Discover the Global University Rankings Dataset 2023 <mark>7 columns</mark>:</h4>
comprehensive and insightful compilation of the world's top universities. Uncover the rankings, metrics, and key performance indicators of renowned academic institutions worldwide. Whether you're a researcher, student, or educator, this dataset provides valuable insights to understand trends and make informed decisions. Explore academic excellence and innovation across diverse disciplines, empowering data-driven analysis and fostering a deeper understanding of the global higher education landscape. Elevate your research and gain a broader perspective with this authoritative resource on university rankings

<a id="1"></a>
# <div style="text-align: center; background-color: #00BFFF; font-family:Times New Roman; color: white; padding: 14px; line-height: 1;border-radius:20px">1. Import Necessary Libraries</div>

In [ ]:
!pip install ydata-profiling

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import shap
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
import missingno as mno
import plotly.offline as pyo 
import plotly.figure_factory as ff
import plotly.io as pio
from wordcloud import WordCloud
color_pal = sns.color_palette()
plt.style.use('seaborn-dark-palette')
plt.style.use('dark_background')

import nltk

import warnings
warnings.filterwarnings('ignore')
sns.set_theme(style='darkgrid', palette='colorblind')
from sklearn.preprocessing import LabelEncoder 
le = LabelEncoder()

#Model
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

<a id="1"></a>
# <div style="text-align: center; background-color: #6495ED; font-family:Times New Roman; color: white; padding: 14px; line-height: 1;border-radius:20px">2. 📊EDA </div>

In [ ]:
df=pd.read_csv('/kaggle/input/global-university-rankings-dataset-2023/world_university_rank.csv')


In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
cols  = df.columns
cols

In [ ]:
df.dtypes

In [ ]:
df.describe().T

In [ ]:
df.describe(include = 'object').T

In [ ]:
df.info()

In [ ]:
unique_values = df.nunique()
unique_values

# <div style="text-align: center; background-color: #6495ED; font-family:Times New Roman; color: white; padding: 14px; line-height: 1;border-radius:20px">3. Null values</div>

In [ ]:
df.isna().sum()

In [ ]:
df.isna().sum().plot(kind ='bar')

In [ ]:
# Fill missing values in the 'locationLocation' column with 'Unknown'
df['locationLocation'].fillna('Unknown', inplace=True)

df.dropna(axis=0,inplace=True)

print(df.shape)

# <div style="text-align: center; background-color: #6495ED; font-family:Times New Roman; color: white; padding: 14px; line-height: 1;border-radius:20px">4. Duplicate rows</div>


In [ ]:
# Finding duplicate rows
duplicate_rows = df[df.duplicated(keep='first')]

# Number of duplicate rows
num_duplicates = duplicate_rows.shape[0]

# Displaying the duplicate rows
print(f"Number of duplicate rows: {num_duplicates}")
duplicate_rows

# <div style="text-align: center; background-color: #6495ED; font-family:Times New Roman; color: white; padding: 14px; line-height: 1;border-radius:20px">5. Feature engineering</div>

In [ ]:
#df['Number of Studnet'] = pd.to_numeric(df['Number of Studnet'], errors='coerce')
df['Number of Studnet'] = df['Number of Studnet'].str.replace(',', '').astype(int) 

In [ ]:
#df['International Student'] = df['International Student'].str.replace('%', '', regex=True).astype(int) 
df['International Student'] = df['International Student'].str.replace('%', ' ')
df['International Student'] = pd.to_numeric(df['International Student'], errors='coerce')

In [ ]:
df['International Student']=  df['International Student']/100

In [ ]:
df.columns = df.columns.str.strip()

# Split the 'Female : Male Ratio' column and create new 'Female' and 'Male' columns
df[['Female', 'Male']] = df['Female : male ratio'].str.split(':', expand=True).astype(int)

# Drop the original 'Female : Male Ratio' column
df.drop(columns=['Female : male ratio'], inplace=True)

# <div style="text-align: center; background-color: #6495ED; font-family:Times New Roman; color: white; padding: 14px; line-height: 1;border-radius:20px">6. Data visualisation</div>

In [ ]:
# Import the ProfileReport class from the ydata_profiling library
from ydata_profiling import ProfileReport

# Create a comprehensive profile report for the DataFrame 'df'
# This report will contain various statistics, insights, and visualizations about the data
profile = ProfileReport(df)

In [ ]:
profile                     # or this one
profile.to_notebook_iframe() # use this line to show the output

In [ ]:
# Calculate the sum of salaries for each job title
monthly_trends = df.groupby('International Student')['Number of Studnet'].sum().reset_index()

# Create a line plot using Plotly Express
# x-axis: International Student, y-axis: Number of Studnet,
# labels for the x-axis, title, and customized height
fig_monthly_trends = px.line(
    monthly_trends,  # DataFrame containing the data
    x='International Student',   # x-values: International Student
    y='Number of Studnet',  # y-values :Number of Studnet
    labels={'International Student': 'International Student'},  # Customize label for the x-axis
    title='Number of Studnet by International Student',  # Set the title of the plot
    height=800  # Set the height of the plot
)

# Display the plot
fig_monthly_trends.show()

In [ ]:
locationLocation_of_Studnet = df.groupby('locationLocation')['Number of Studnet'].sum().reset_index()

# Create a line plot using Plotly Express
# x-axis: locationLocation, y-axis: Number of Studnet ,
# labels for the x-axis, title, and customized height
locationLocation_of_Studnet = px.line(
    locationLocation_of_Studnet,  # DataFrame containing the data
    x='locationLocation',   # x-values
    y='Number of Studnet',  # y-values
    labels={'job_title': 'Job'},  # Customize label for the x-axis
    title='Number of Studnet  by location',  # Set the title of the plot
    height=800  # Set the height of the plot
)

# Display the plot
locationLocation_of_Studnet.show()

In [ ]:
Number_of_Female_Studnet = df.groupby('Female')['Number of Studnet'].sum().reset_index()

# Create a line plot using Plotly Express
# x-axis: locationLocation, y-axis: Number of Studnet ,
# labels for the x-axis, title, and customized height
Number_of_Female_Studnet = px.line(
    Number_of_Female_Studnet,  # DataFrame containing the data
    x='Female',   # x-values
    y='Number of Studnet',  # y-values
    labels={'Female': 'Female'},  # Customize label for the x-axis
    title='Number of Studnet  by Female Number',  # Set the title of the plot
    height=800  # Set the height of the plot
)

# Display the plot
Number_of_Female_Studnet.show()

In [ ]:
Number_of_Female_Studnet = df.groupby(['Female','Male'])['Number of Studnet'].sum().reset_index()

# Create a line plot using Plotly Express
# x-axis: locationLocation, y-axis: Number of Studnet ,
# labels for the x-axis, title, and customized height
Number_of_Female_Studnet = px.line(
    Number_of_Female_Studnet,  # DataFrame containing the data
    x='Female',   # x-values
    y='Number of Studnet',  # y-values
    labels={'Female': 'Female , Male '},  # Customize label for the x-axis
    title='Number of Studnet  by Female , Male Number',  # Set the title of the plot
    height=800  # Set the height of the plot
)

# Display the plot
Number_of_Female_Studnet.show()

In [ ]:
# Calculate the value counts for each unique value in the 'job_title' column
top10_job_title = df['locationLocation'].value_counts()[:10]

# Create a bar plot using Plotly Express
# y-axis: Count of each job title, x-axis: Job titles,
# color-coded by job titles and using a color palette
fig = px.bar(
    y=top10_job_title.values,  # y-values: count of each job title
    x=top10_job_title.index,   # x-values: job titles
    color=top10_job_title.index,  # Color the bars based on job titles
    color_discrete_sequence=px.colors.sequential.PuBuGn,  # Set color palette
    text=top10_job_title.values,  # Display the count values on top of the bars
    title='Top 10 locationL ',  # Set the title of the plot
    template='plotly_dark'  # Use a dark template for the plot
)

# Update the layout of the plot
fig.update_layout(
    title_text='location Distribution',  # Set the title of the plot
    height=650,  # Set the height of the plot
    xaxis_title="location",  # Label for the x-axis
    yaxis_title="Count",  # Label for the y-axis
    font=dict(size=17, family="Franklin Gothic")  # Set the font size and family for the text
)

# Display the plot
fig.show()

In [ ]:
# Calculate the value counts for each unique value in the 'job_title' column
top10_Number_of_Studnet = df['Number of Studnet'].value_counts()[:10]

# Create a bar plot using Plotly Express
# y-axis: Count of each Number of Studnet , x-axis: Number of Studnet,
# color-coded by Number of Studnet and using a color palette
fig = px.bar(
    y=top10_Number_of_Studnet.values,  # y-values: count of each Number of Studnet
    x=top10_Number_of_Studnet.index,   # x-values: Number of Studnet
    color=top10_Number_of_Studnet.index,  # Color the bars based on job titles
    color_discrete_sequence=px.colors.sequential.PuBuGn,  # Set color palette
    text=top10_Number_of_Studnet.values,  # Display the count values on top of the bars
    title='Top 10 Number of Studnet ',  # Set the title of the plot
    template='plotly_dark'  # Use a dark template for the plot
)

# Update the layout of the plot
fig.update_layout(
    title_text='Number of Studnet Distribution',  # Set the title of the plot
    height=700,  # Set the height of the plot
    xaxis_title="Number of Studnet",  # Label for the x-axis
    yaxis_title="Count",  # Label for the y-axis
    font=dict(size=17, family="Franklin Gothic")  # Set the font size and family for the text
)

# Display the plot
fig.show()

In [ ]:
# Select the top 5 salaries in the DataFrame
top5_Number_of_Studnet = df['Number of Studnet'].sort_values(ascending=False).head(5)

# Create a bar plot using Plotly Express
# y-axis: S=Number of Studnet, x-axis: Index of the top 5 Number of Studnet,
# color-coded by the index, and using a color palette
fig = px.bar(
    y=top5_Number_of_Studnet.values,  # y-values: top 5 Number of Studnet
    x=top5_Number_of_Studnet.index,   # x-values: indices of the top 5 Number of Studnet
    color=top5_Number_of_Studnet.index,  # Color the bars based on the indices
    color_discrete_sequence=px.colors.sequential.PuBuGn,  # Set color palette
    text=top5_Number_of_Studnet.values,  # Display the salary values on top of the bars
    title='Top 5 Number of Studnet',  # Set the title of the plot
    template='plotly_dark'  # Use a dark template for the plot
)

# Update the layout of the plot
fig.update_layout(
    title_text='Number of Studnet Distribution',  # Set the title of the plot
    height=650,  # Set the height of the plot
    xaxis_title="Number of Studnet",  # Label for the x-axis
    yaxis_title="Count",  # Label for the y-axis
    font=dict(size=17, family="Franklin Gothic")  # Set the font size and family for the text
)

# Display the plot
fig.show()

In [ ]:
# x-axis: "locationLocation", y-axis: "Number of student per staffs", color-coded by "Number of Studnet"
fig = px.bar(df, x="locationLocation", y="Number of student per staffs", color="Number of Studnet")

# Update the layout of the plot
fig.update_layout(
    title_text='locationLocation Distribution with Number of student per staffs ',  # Set the title of the plot
    height=650,  # Set the height of the plot
    xaxis_title="locationLocation",  # Label for the x-axis
    yaxis_title="Number of student per staffs",  # Label for the y-axis
    font=dict(size=17, family="Franklin Gothic")  # Set the font size and family for the text
)

# Display the plot
fig.show()

In [ ]:
# x-axis: "locationLocation", y-axis: "International Student", color-coded by "Number of Studnet"
fig = px.bar(df, x="locationLocation", y="International Student", color="Number of Studnet")

# Update the layout of the plot
fig.update_layout(
    title_text='locationLocation Distribution with  International Student',  # Set the title of the plot
    height=650,  # Set the height of the plot
    xaxis_title="locationLocation",  # Label for the x-axis
    yaxis_title="International Student",  # Label for the y-axis
    font=dict(size=17, family="Franklin Gothic")  # Set the font size and family for the text
)

# Display the plot
fig.show()

In [ ]:
# x-axis: "locationLocation", y-axis: "Number of Studnet", color-coded by "Female"
fig = px.bar(df, x="Female", y="Number of Studnet", color="Female")

# Update the layout of the plot
fig.update_layout(
    title_text='Number of Studnet Distribution with  Female Student',  # Set the title of the plot
    height=650,  # Set the height of the plot
    xaxis_title="Female",  # Label for the x-axis
    yaxis_title="Number of Studnet",  # Label for the y-axis
    font=dict(size=17, family="Franklin Gothic")  # Set the font size and family for the text
)

# Display the plot
fig.show()

In [ ]:
# x-axis: "Male", y-axis: "Number of Studnet", color-coded by "Male"
fig = px.bar(df, x="Male", y="Number of Studnet", color="Male")

# Update the layout of the plot
fig.update_layout(
    title_text='Number of Studnet Distribution with  Male Student',  # Set the title of the plot
    height=650,  # Set the height of the plot
    xaxis_title="Male",  # Label for the x-axis
    yaxis_title="Number of Studnet",  # Label for the y-axis
    font=dict(size=17, family="Franklin Gothic")  # Set the font size and family for the text
)

# Display the plot
fig.show()

In [ ]:
def Freq_df(cleanwordlist):
    Freq_dist_nltk = nltk.FreqDist(cleanwordlist)
    df_freq = pd.DataFrame.from_dict(Freq_dist_nltk, orient='index')
    df_freq.columns = ['Frequency']
    df_freq.index.name = 'Term'
    df_freq = df_freq.sort_values(by=['Frequency'],ascending=False)
    df_freq = df_freq.reset_index()
    return df_freq

def Word_Cloud(data, color_background, colormap, title):
    plt.figure(figsize = (20,15))
    wc = WordCloud(width=1200, 
               height=600, 
               max_words=50,
               colormap= colormap,
               max_font_size = 100,
               random_state=88, 
               background_color=color_background).generate_from_frequencies(data)
    plt.imshow(wc, interpolation='bilinear')
    plt.title(title, fontsize=20)
    plt.axis('off')
    plt.show()


In [ ]:
freq_df = Freq_df(df['University name'].values.tolist())
data = dict(zip(freq_df['Term'].tolist(), freq_df['Frequency'].tolist()))
data = freq_df.set_index('Term').to_dict()['Frequency']

Word_Cloud(data ,'black','RdBu', 'WordCloud of University name')

In [ ]:
# Box Plot
# Prepare data for the box plot
hist_data = [df['Number of Studnet']]
group_labels = ['Number of Studnet']

# Create the box plot using Plotly Express
fig1 = px.box(y=df['Number of Studnet'], template='plotly_dark', title='Number of Studnet (BoxPlot)')

# Customize the layout of the box plot
fig1.update_layout(font=dict(size=17, family="Franklin Gothic"))

# Display the box plot
fig1.show()

# Dist Plot
# Prepare data for the distribution plot
hist_data = [df['Number of Studnet']]
group_labels = ['Number of Studnet']

# Create the distribution plot using Plotly Figure Factory
fig2 = ff.create_distplot(hist_data, group_labels, show_hist=False)
fig2.layout.template = 'plotly_dark'

# Customize the layout of the distribution plot
fig2.update_layout(title='Number of Studnet (DistPlot)', font=dict(size=17, family="Franklin Gothic"))

# Display the distribution plot
fig2.show()


In [ ]:
# Identify outliers in 'Number of Studnet' column using the IQR method on scaled data
Q1 = df['Number of Studnet'].quantile(0.25)
Q3 = df['Number of Studnet'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

outliers = df[(df['Number of Studnet'] < lower_bound) | (df['Number of Studnet'] > upper_bound)]

print("Number of outliers:", len(outliers))

# Remove outliers from the DataFrame
df = df[(df['Number of Studnet'] >= lower_bound) & (df['Number of Studnet'] <= upper_bound)]

In [ ]:
# Remove rows with NaN values in the 'International Student' column
df_cleaned = df.dropna(subset=['International Student'])

# Box Plot
fig1 = px.box(df_cleaned, y='International Student', template='plotly_dark', title='International Student (BoxPlot)')
fig1.update_layout(font=dict(size=17, family="Franklin Gothic"))
fig1.show()

# Dist Plot
hist_data = [df_cleaned['International Student']]
group_labels = ['International Student']
fig2 = ff.create_distplot(hist_data, group_labels, show_hist=False)
fig2.layout.template = 'plotly_dark'
fig2.update_layout(title='International Student (DistPlot)', font=dict(size=17, family="Franklin Gothic"))
fig2.show()


In [ ]:
# Remove rows with NaN values in the 'Number of student per staffs' column
# Box Plot
fig1 = px.box(df_cleaned, y='Number of student per staffs', template='plotly_dark', title='Number of student per staffs (BoxPlot)')
fig1.update_layout(font=dict(size=17, family="Franklin Gothic"))
fig1.show()

# Dist Plot
hist_data = [df_cleaned['Number of student per staffs']]
group_labels = ['Number of student per staffs']
fig2 = ff.create_distplot(hist_data, group_labels, show_hist=False)
fig2.layout.template = 'plotly_dark'
fig2.update_layout(title='Number of student per staffs (DistPlot)', font=dict(size=17, family="Franklin Gothic"))
fig2.show()


In [ ]:
# Identify outliers in 'Number of student per staffs' column using the IQR method on scaled data
Q1 = df['Number of student per staffs'].quantile(0.25)
Q3 = df['Number of student per staffs'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

outliers = df[(df['Number of student per staffs'] < lower_bound) | (df['Number of student per staffs'] > upper_bound)]

print("Number of outliers:", len(outliers))

# Remove outliers from the DataFrame
df = df[(df['Number of student per staffs'] >= lower_bound) & (df['Number of student per staffs'] <= upper_bound)]

In [ ]:
# Calculate the locationLocation_International_Student for each job title and sort in descending order
locationLocation_International_Student = df.groupby('locationLocation')['International Student'].mean().reset_index()
locationLocation_International_Student = locationLocation_International_Student.sort_values(by='International Student', ascending=False)

# Select the top 10 job titles with the highest average salary
top_10_locationLocation_International_Student = locationLocation_International_Student.head(10)

# Create a new figure using Plotly
fig = go.Figure()

# Add a bar trace to the figure
fig.add_trace(go.Bar(
    x=top_10_locationLocation_International_Student['locationLocation'],  # x-values: locationLocation
    y=top_10_locationLocation_International_Student['International Student']  # y-values: International Student
))

# Update the layout and appearance of the plot
fig.update_layout(
    title='Top 10 International Student by location',  # Set the title of the plot
    xaxis_title='location',  # Label for the x-axis
    yaxis_title='International Student',  # Label for the y-axis
    paper_bgcolor='#595964',  # Background color of the plot
    template='plotly_dark',  # Use a dark template for the plot
    font=dict(color='white'),  # Set font color to white
    height=650  # Set the height of the plot
)

# Initialize Plotly for notebook integration
pyo.init_notebook_mode(connected=True)

# Display the plot
pyo.iplot(fig)


In [ ]:
v = px.funnel(df[:20], x="International Student", y="locationLocation")
v.show()

In [ ]:
# x-axis: "University name", y-axis: "Number of Studnet", color-coded by "experience_level"
fig = px.scatter(df, x="University name", y="Number of Studnet", color="Number of Studnet")

# Update the layout of the plot
fig.update_layout(
    title_text='Number of Studnet Distribution',  # Set the title of the plot
    height=1000,  # Set the height of the plot
    xaxis_title="University name",  # Label for the x-axis
    yaxis_title="Number of Studnet",  # Label for the y-axis
    font=dict(size=17, family="Franklin Gothic")  # Set the font size and family for the text
)

# Display the plot
fig.show()

In [ ]:
# x-axis: "University name", y-axis: "Number of stude\nt per staffs	", color-coded by "experience_level"
fig = px.scatter(df, x="University name", y="Number of student per staffs", color="Number of student per staffs")

# Update the layout of the plot
fig.update_layout(
    title_text='Number of student per staffs Distribution',  # Set the title of the plot
    height=1000,  # Set the height of the plot
    xaxis_title="University name",  # Label for the x-axis
    yaxis_title="Number of student per staffs",  # Label for the y-axis
    font=dict(size=17, family="Franklin Gothic")  # Set the font size and family for the text
)

# Display the plot
fig.show()

In [ ]:
# x-axis: "University name", y-axis: "International Student", color-coded by "experience_level"
fig = px.scatter(df, x="University name", y="International Student", color="International Student")

# Update the layout of the plot
fig.update_layout(
    title_text='International Student Distribution',  # Set the title of the plot
    height=1000,  # Set the height of the plot
    xaxis_title="University name",  # Label for the x-axis
    yaxis_title="International Student",  # Label for the y-axis
    font=dict(size=17, family="Franklin Gothic")  # Set the font size and family for the text
)

# Display the plot
fig.show()

In [ ]:
# Creating the choropleth map
fig = px.choropleth(
    df,
    locations='locationLocation',  # Replace with the actual column name for locations
    locationmode='country names',  # Assuming your location names are country names
    color='Number of Studnet',  # Color based on the number of students
    color_continuous_scale=px.colors.sequential.Peach,
    template='plotly_dark',
    hover_name='University name',  # Displayed when hovering over each location
    hover_data={'Number of Studnet': True},  # Additional data to display in the hover tooltip
    title='Universities with Students'
)

# Customize hover template to display university name and number of students
fig.update_traces(
    hovertemplate='<b>%{hovertext}</b><br>Number of Studnet: %{customdata[0]}'
)

# Show the map
fig.show()


# <div style="text-align: center; background-color: #6495ED; font-family:Times New Roman; color: white; padding: 14px; line-height: 1;border-radius:20px">7. Categorical</div>

In [ ]:
df['locationLocation'] = le.fit_transform(df['locationLocation'])
df['University name'] = le.fit_transform(df['University name'])

In [ ]:
df=df.dropna()

# <div style="text-align: center; background-color: #6495ED; font-family:Times New Roman; color: white; padding: 14px; line-height: 1;border-radius:20px">8. Corr Matrix
</div>

In [ ]:
Corr_Matrix = df.corr()

# Set up the figure and plot the heatmap
plt.figure(figsize=(10, 10))
sns.heatmap(Corr_Matrix, annot=True, cmap='coolwarm', center=0)
plt.show()

In [ ]:
print('Top 5 Most Positively Correlated to the To salary_in_usd')
Corr_Matrix['International Student'].sort_values(ascending=False).head(5)

In [ ]:
print('Top 5 Most Negatively Correlated to salary_in_usd ')
Corr_Matrix['International Student'].sort_values(ascending=True).head(5)

# <div style="text-align: center; background-color: #6495ED; font-family:Times New Roman; color: white; padding: 14px; line-height: 1;border-radius:20px">9. spliting the dataset
</div>

In [ ]:
X = df.drop(columns=['Rank','University name','International Student'])
y = df['International Student']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Display the shapes of the resulting datasets
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

# <div style="text-align: center; background-color: #6495ED; font-family:Times New Roman; color: white; padding: 14px; line-height: 1;border-radius:20px">10. Model Building and Analysis

</div>

In [ ]:
models = {
    'Random Forest': RandomForestRegressor(random_state=42),
    'Gradient Boosting': GradientBoostingRegressor(random_state=42),
}
best_model = None
best_r2 = 0

for model_name, model in models.items():
    model.fit(X_train, y_train)
    y_pred= model.predict(X_test)

    # Evaluate the model
    r2 = r2_score(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    submit = pd.DataFrame()
    submit['Actual_salary'] = y_test
    submit['Predict_salary'] = y_pred
    submit = submit.reset_index()
    r2 = r2_score(y_test, y_pred)
    if r2 > best_r2:
        best_r2 = r2
        best_model = model.__class__.__name__

    print(f'{model_name}:')
    print(f'R2 Score: {r2:.2f}')
    print(f'Mean Absolute Error (MAE): {mae:.2f}')
    print(f'Root Mean Squared Error (RMSE): {rmse:.2f}')
    print(submit.head(5))

    print('----------------------------------------')
print(f"The best performing model is: {best_model} with accuracy: {best_r2:.2f}")

# <div style="text-align: center; background-color: #6495ED; font-family:Times New Roman; color: white; padding: 14px; line-height: 1;border-radius:20px">11. feature importances

</div>

In [ ]:
importances = model.feature_importances_

feature_names = X.columns

feature_importance_dict = dict(zip(feature_names, importances))

sorted_feature_importance = sorted(feature_importance_dict.items(), key=lambda x: x[1], reverse=True)

for feature, importance in sorted_feature_importance:
    print(f"{feature}: {importance:.2f}")

plt.figure(figsize=(12, 7))
plt.barh(*zip(*sorted_feature_importance), alpha=0.9, color='teal')
plt.title('Feature Importance', fontsize=15)
plt.xlabel("Importance")
plt.ylabel("Feature")
plt.show()

In [ ]:
y_pred= model.predict(X_test)

# Residuals
residuals = y_test - y_pred

# Plot
plt.figure(figsize=(10, 6))
sns.scatterplot(x=y_pred, y=residuals)
plt.axhline(y=0, color='r', linestyle='--')
plt.xlabel('Predicted Values')
plt.ylabel('Residuals')
plt.title('Residual Plot')
plt.show()